# Install PEcAn with helm

Run the commands below to install PEcAn for ecological forecasting. 

In [ ]:
%%bash
git clone https://github.com/computate-org/pecan-helm.git ~/.local/src/pecan-helm/
helm repo add ncsa https://opensource.ncsa.illinois.edu/charts/
(cd ~/.local/src/pecan-helm/ && helm dependency build)
helm upgrade --install pecan ~/.local/src/pecan-helm/  \
  --set betydb.enabled=false \
  --set betydb.postgresql.enabled=false \
  --set betydb.postgresql.postgresqlHost=pecan-postgresql \
  --set betydb.postgresql.postgresqlUser=postgres \
  --set betydb.postgresql.postgresqlUsername=postgres \
  --set betydb.postgresql.postgresqlDatabase=postgres \
  --set betydb.postgresql.service.port=5432 \
  --set betydb.service.port=8000 \
  --set betydb.betyUser=bety \
  --set betydb.betyDatabase=bety \
  --set betydb.ingress.hosts={pecan-betydb} \
  --set rabbitmq.enabled=false \
  --set rabbitmq.rabbitmq.username=guest \
  --set rabbitmq.rabbitmq.password=guest \
  --set rabbitmq.rabbitmq.erlangCookie=1234thisisasupersecretoflength32 \
  --set rabbitmq.auth.username=guest \
  --set rabbitmq.auth.password=guest \
  --set rabbitmq.auth.erlangCookie=1234thisisasupersecretoflength32 \
  --set rabbitmq.ingress.enabled=false \
  --set rabbitmq.ingress.hostName=rabbitmq.localhost \
  --set rabbitmq.ingress.path=/ \
  --set clustername=pecan-server \
  --set clusterfqdn=pecan.$(oc get route python -o jsonpath={.spec.host} | cut -d "." -f2-) \
  --set initializeData=true \
  --set ingress.enabled=true \
  --set pecan.env.betydb=http://betydb:8000/ \
  --set persistence.size=3Gi \
  --set persistence.accessMode=ReadWriteOnce \
  --set docs.replicaCount=0 \
  --set executor.replicaCount=0 \
  --set models.biocro-0-95.replicaCount=0 \
  --set models.ed2-2-2-0.replicaCount=0 \
  --set models.maespa-git.replicaCount=0 \
  --set models.sipnet-git.replicaCount=0 \
  --set models.sipnet-136.replicaCount=0 \
  --set web.replicaCount=0 \
  --set web.service.port=8080 \
  --set ingress.hosts={pecan.$(oc get route python -o jsonpath={.spec.host} | cut -d "." -f2-)}
echo DONE

## ReadWriteOnce PersistentVolumeClaims

Because the current state of the PEcAn platform depends on the availability of Read/Write Many Persistent Volume Claims, 
which is not an available feature of the Red Hat OpenShift Developer Sandbox, 
we need to work around this limitation. 
PEcAn sets up the executor, web, and model pods to all mount the same data volume. 
The Developer Sandbox only supports Read/Write Once PVCs instead which sometimes blocks all of these services from running if they are on different nodes. 

To work around this issue, we will first deploy the PEcAn executor service, and then deploy web and model pods on the same node. 

### Check the pecan-load-data Job

The `pecan-load-data` Job initializes the database. 
When the job is complete, it will be deleted. 

In [ ]:
%%bash
oc wait job -l app.kubernetes.io/name=pecan-load-data --for delete
echo DONE

### Deploy the PEcAn executor after the pecan-load-data Job

In [ ]:
%%bash
oc scale deployment/pecan-executor --replicas 1
oc get pod -l app.kubernetes.io/name=pecan-executor
oc wait pod -l app.kubernetes.io/name=pecan-executor --for=condition=Ready --timeout=2m
oc get pod -l app.kubernetes.io/name=pecan-executor
echo DONE

### Update the node name of the pecan-executor deployment

In [ ]:
%%bash
PECAN_DATA_NODE=$(oc get $(oc get pod -l app.kubernetes.io/name=pecan-executor -o name) -o jsonpath={.spec.nodeName})
echo $PECAN_DATA_NODE
oc patch deployment/pecan-executor --type merge \
  -p '{"spec":{"template":{"spec":{"nodeName":"'"$PECAN_DATA_NODE"'"}}}}'
oc get pod -l app.kubernetes.io/name=pecan-executor
oc wait pod -l app.kubernetes.io/name=pecan-executor --for=condition=Ready --timeout=2m
oc get pod -l app.kubernetes.io/name=pecan-executor
echo DONE

### Deploy the sipnet-136 model on the same data node

In [ ]:
%%bash
PECAN_DATA_NODE=$(oc get $(oc get pod -l app.kubernetes.io/name=pecan-executor -o name) -o jsonpath={.spec.nodeName})
echo $PECAN_DATA_NODE
oc patch deployment/pecan-model-sipnet-136 --type merge \
  -p '{"spec":{"template":{"spec":{"nodeName":"'"$PECAN_DATA_NODE"'"}}}}'
oc scale deployment/pecan-model-sipnet-136 --replicas 1
oc get pod -l app.kubernetes.io/name=pecan-model-sipnet-136
oc wait pod -l app.kubernetes.io/name=pecan-model-sipnet-136 --for=condition=Ready --timeout=2m
oc get pod -l app.kubernetes.io/name=pecan-model-sipnet-136
echo DONE

### Deploy the sipnet-git model on the same data node

In [ ]:
%%bash
PECAN_DATA_NODE=$(oc get $(oc get pod -l app.kubernetes.io/name=pecan-executor -o name) -o jsonpath={.spec.nodeName})
echo $PECAN_DATA_NODE
oc patch deployment/pecan-model-sipnet-git --type merge \
  -p '{"spec":{"template":{"spec":{"nodeName":"'"$PECAN_DATA_NODE"'"}}}}'
oc scale deployment/pecan-model-sipnet-git --replicas 1
oc get pod -l app.kubernetes.io/name=pecan-model-sipnet-git
oc wait pod -l app.kubernetes.io/name=pecan-model-sipnet-git --for=condition=Ready --timeout=2m
oc get pod -l app.kubernetes.io/name=pecan-model-sipnet-git
echo DONE

### Deploy the PEcAn Web application on the same data node

In [ ]:
%%bash
PECAN_DATA_NODE=$(oc get $(oc get pod -l app.kubernetes.io/name=pecan-executor -o name) -o jsonpath={.spec.nodeName})
echo $PECAN_DATA_NODE
oc patch deployment/pecan-web --type merge \
  -p '{"spec":{"template":{"spec":{"nodeName":"'"$PECAN_DATA_NODE"'"}}}}'
oc scale deployment/pecan-web --replicas 1
oc get pod -l app.kubernetes.io/name=pecan-web
oc wait pod -l app.kubernetes.io/name=pecan-web --for=condition=Ready --timeout=2m
oc get pod -l app.kubernetes.io/name=pecan-web
echo DONE

## Discover the PEcAn web service

Now discover the URL to the running instance of the PEcAn Web service. 
Discover the route to the PEcAn web service by running the command below. 

![PEcAn Web service](pictures/pecan-web-01.png)

In [ ]:
%%bash
oc get $(oc get route -o name | grep pecan-web) -o jsonpath={.spec.host} | xargs -0 printf 'http://%s/pecan/\n'
echo DONE

Click [ Next ] to move to the next page where you can select an Ecological Forecasting host, model and site. 

![PEcAn map](pictures/pecan-web-02.png)

Start with some common defaults for this mode run: 

- Host: `pecan-server`
- Model: `SIPNET (r136)`
- Site Group: `AmeriFlux`

![Select host, model, and site group](pictures/pecan-web-03.png)

Click on the waypoints that appear on the map and find a location and click on it. 
Then click [ Next ]. 

![Find site](pictures/pecan-web-04.png)

Choose a PFT, like `boreal.coniferous`. 
Then click [ Next ]. 

![Choose a PFT](pictures/pecan-web-05.png)

Now watch the model run! 

![Watch the model run](pictures/pecan-web-06.png)